# Studi Kasus Beer Production

## PROFILE KELOMPOK


NPM KETUA : 200411100153

NAMA KETUA : FIKRI AINUN NAJIB

NPM ANGGOTA : 200411100166

NAMA ANGGOTA : GHANIY ALBAR RASYIDI KUSUMA 

GITHUB : https://github.com/LALA09-erha/kolaborasipro

DATASET : https://www.kaggle.com/datasets/shenba/time-series-datasets?select=monthly-beer-production-in-austr.csv

MODEL : KNN REGRESI , GAUSSIAN REGRESI , bagging Regressor > SVR , random forest Regressor , stacking Regressor > [RidgeCV ,LinearSVR] 

KELAS PSD C

Inisialisasi Libary dan beberapa Model

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import array
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR

Pengambilan data

In [2]:
data = pd.read_csv("../Data/BKDP.JK.csv")

# add new data input to the dataframe
# data = data.append({'Month': '1995-09', 'Monthly beer production': 0}, ignore_index=True)

data = data['Volume']

Split Data

In [3]:
data_training,data_test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=False, random_state=42)

In [4]:
data_training_new = pd.DataFrame(data_training)
data_test_new = pd.DataFrame(data_test)

Preprosessing Data

In [5]:
sc = MinMaxScaler()
train_scaled = sc.fit_transform(data_training_new)
test_scaled = sc.transform(data_test_new)

fungsi Cross Validation

In [6]:
# Cross Validation
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  
  return array(X), array(y)
# define univariate time series

Eksekusi Model

In [7]:
n_steps = range(1,5)
acc_fix = {}

for n in n_steps:
    # Proses Pemanggilan Cross Validation 
    X_train, y_train = split_sequence(train_scaled, n) 
    X_test, y_test = split_sequence(test_scaled, n) 

    # membuat kolom otomatis
    colom = []
    for c in range(n):
        if(c==0):
            colom.insert(0,"Xt")
        else:
            colom.insert(0,f"Xt-{c}")

    # # Masukkan ke dataframe hasil dari split  squens
    # UNTUK MEMISAHKAN X TRAIN, YTRAIN dan TEST
    newX_train = pd.DataFrame(np.reshape(X_train, (len(X_train) , n)),columns=colom)
    newY_train = pd.DataFrame(y_train, columns=['Output'])
    new_data_train = pd.concat([newX_train,newY_train],axis=1)

    newX_test = pd.DataFrame(np.reshape(X_test, (len(X_test) , n)),columns=colom)
    newY_test = pd.DataFrame(y_test, columns=['Output'])
    new_data_test = pd.concat([newX_test,newY_test],axis=1)


    # EKSEKUSI KE model

    # predict kkn
    k_nei = range(1,30)
    acc = []
    for k in k_nei:
        knn = KNeighborsRegressor(n_neighbors = k)
        knn.fit(newX_train,newY_train)
        y_pred = knn.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = sc.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = sc.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc.append(mape)
    
    # masukkan ke acc-fix 
    acc_fix[f"n-{n} kkn"] = [min(acc) , acc.index(min(acc))+1]
        
    # predict gaussian regressor
    kernel = DotProduct() + WhiteKernel()
    gpr = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(newX_train, newY_train)
    y_pred = gpr.predict(newX_test, return_std=True)[0]
    y_pred_shape = y_pred.reshape(-1,1)
    y_pred_inverse = sc.inverse_transform(y_pred_shape)
    newY_test_shape = y_test.reshape(-1,1)
    newY_test_inverse = sc.inverse_transform(newY_test_shape)
    # print(y_pred_inverse)
    mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)

    acc_fix[f"n-{n} gpr"] = [mape]

    # Predict SVR dengan Bagging Regressor
    n_estimators = range(1,30)
    acc_svr = []
    for e in n_estimators:
        regr = BaggingRegressor(SVR(), n_estimators=e, random_state=0).fit(newX_train, newY_train.values.ravel())
        y_pred = regr.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = sc.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = sc.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc_svr.append(mape)
    
    # masukkan ke acc-fix
    acc_fix[f"n-{n} bagsvr"] = [min(acc_svr) , acc_svr.index(min(acc_svr))+1]

    # Predict dengan from RandomForestRegressor
    maxd = range(1,10)
    acc_rf = []
    for d in maxd:
        regr = RandomForestRegressor(max_depth=d, random_state=0).fit(newX_train, newY_train.values.ravel())
        y_pred = regr.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = sc.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = sc.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc_rf.append(mape)
    
    # masukkan ke acc-fix
    acc_fix[f"n-{n} rf"] = [min(acc_rf) , acc_rf.index(min(acc_rf)) +1]

    # Prdict dengan LinearSVR , Ridge pada stacking regressors
    estimators = [('lr', RidgeCV()),
                ('svr', LinearSVR( random_state=42, max_iter=100000))]
    acc_stac = []
    for e in n_estimators:
        reg = StackingRegressor(
            estimators,
            final_estimator=RandomForestRegressor(n_estimators=e,
                                                random_state=42))
        reg.fit(newX_train, newY_train.values.ravel())
        y_pred = reg.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = sc.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = sc.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc_stac.append(mape)
    
    # masukkan ke acc-fix
    acc_fix[f"n-{n} stac"] = [min(acc_stac) , acc_stac.index(min(acc_stac)) +1]
 


### Hasil Berbagai Model

Menentukan MAPE terendah dari Berbagai Model Diatas

In [8]:
temp = 1
temp2 = {}
for m in acc_fix:
    if(acc_fix[m][0] < temp ):
        temp = acc_fix[m][0]
        temp2 = {}
        temp2[m] = acc_fix[m]
print(f"Hasil Dari Berbagai Model yang memiliki MAPE terkecil adalah: {temp2}")
        


Hasil Dari Berbagai Model yang memiliki MAPE terkecil adalah: {'n-1 rf': [0.7201551440061844, 3]}
